In [2]:
import torch
import torchaudio
import sys
import os
import requests
import torchaudio
from torchaudio.compliance import kaldi
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image
import importlib
import models_mae
import librosa
import librosa.display
from dataset import AudiosetDataset 

ModuleNotFoundError: No module named 'timm.models.swin_transformer'

In [4]:
!conda list

# packages in environment at /data/scratch/ngop/AudioMAE_env:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                        main  
_openmp_mutex             4.5                       1_gnu  
absl-py                   1.0.0                    pypi_0    pypi
anyio                     3.6.1                    pypi_0    pypi
appdirs                   1.4.4                    pypi_0    pypi
argon2-cffi               21.3.0                   pypi_0    pypi
argon2-cffi-bindings      21.2.0                   pypi_0    pypi
asttokens                 2.0.5                    pypi_0    pypi
attr                      0.3.1                    pypi_0    pypi
attrs                     21.4.0                   pypi_0    pypi
audioread                 2.1.9                    pypi_0    pypi
av                        8.1.0                    pypi_0    pypi
babel                     2.10.1                   pypi_0    pypi
backcall               